# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [8]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_test = pd.read_csv('data/y_test.csv')
y_val = pd.read_csv('data/y_val.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

Here, we're calling .shape on our training data so that we can use the result as n_features, so we know how big to make our input layer.

In [5]:
# How big input layer?
n_features = (X_train.shape[1],)
print(n_features)

(296,)


Create your baseline model. Yo will notice is exihibits strange behavior.

*Note:* When you run this model or other models later on, you may get a comment from tf letting you about optimizing your GPU

In [9]:
# Baseline model
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(Dense(100, activation='relu',
                         input_shape=(X_train.shape[1],)))

# Hidden layer with 50 units
baseline_model.add(Dense(50, activation='relu'))  

# Output layer
baseline_model.add(Dense(1, activation='sigmoid'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Epoch 1/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 40829464576.0000 - mse: 40829464576.0000 - val_loss: 36092796928.0000 - val_mse: 36092796928.0000
Epoch 2/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 39130619904.0000 - mse: 39130619904.0000 - val_loss: 36092796928.0000 - val_mse: 36092796928.0000
Epoch 3/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 38760009728.0000 - mse: 38760009728.0000 - val_loss: 36092796928.0000 - val_mse: 36092796928.0000
Epoch 4/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 40865726464.0000 - mse: 40865726464.0000 - val_loss: 36092796928.0000 - val_mse: 36092796928.0000
Epoch 5/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 40467001344.0000 - mse: 40467001344.0000 - val_loss: 36092796928.0000 - val_mse: 36092796928.0000
Epoch 6/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 40438104064.0000 - mse: 40438104064.0000 - val_loss: 36092796928.0000 - val_mse: 36092796928.0000
Epoch 7/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss:

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [11]:
# Numeric column names
numeric_columns = X_train_numeric.columns.tolist() 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(
    ss_X.fit_transform(X_train_numeric), 
    columns=numeric_columns
)

# Transform validate and test data
X_val_scaled = pd.DataFrame(
    ss_X.transform(X_val_numeric), 
    columns=numeric_columns
)
X_test_scaled = pd.DataFrame(
    ss_X.transform(X_test_numeric), 
    columns=numeric_columns
)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [ ]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 
Again, you may get some strange behavior.

In [12]:
# Train the model
normalized_input_model = Sequential()
normalized_input_model.add(Dense(100, activation='relu', input_shape=(X_train.shape[1],)))
normalized_input_model.add(Dense(50, activation='relu'))
normalized_input_model.add(Dense(1, activation='sigmoid'))
#Compile the model
normalized_input_model.compile(optimizer='SGD',
                                 loss='binary_crossentropy', 
                                 metrics=['acc'])

# Train the model
normalized_input_model.fit(X_train, 
                            y_train, 
                            batch_size=32, 
                            epochs=150, 
                            validation_data=(X_val, y_val))

Epoch 1/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - acc: 0.0000e+00 - loss: nan - val_acc: 0.0000e+00 - val_loss: nan
Epoch 2/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.0000e+00 - loss: nan - val_acc: 0.0000e+00 - val_loss: nan
Epoch 3/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.0000e+00 - loss: nan - val_acc: 0.0000e+00 - val_loss: nan
Epoch 4/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.0000e+00 - loss: nan - val_acc: 0.0000e+00 - val_loss: nan
Epoch 5/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.0000e+00 - loss: nan - val_acc: 0.0000e+00 - val_loss: nan
Epoch 6/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.0000e+00 - loss: nan - val_acc: 0.0000e+00 - val_loss: nan
Epoch 7/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.0000e+00 - loss: nan - val_acc: 0.0000e+00 - val_loss: nan
Epoch 8/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.0000e+00 - loss: nan - val_acc: 0.0000e+00 - val_loss: nan
Epoch 9/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/st

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [13]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [20]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
history = normalized_model.fit(X_train,
                               y_train_scaled, 
                               batch_size=32, 
                               epochs=150, 
                               validation_data=(X_val, y_val_scaled))


Epoch 1/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5950 - mse: 0.5950 - val_loss: 0.1820 - val_mse: 0.1820
Epoch 2/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2167 - mse: 0.2167 - val_loss: 0.1391 - val_mse: 0.1391
Epoch 3/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2051 - mse: 0.2051 - val_loss: 0.1210 - val_mse: 0.1210
Epoch 4/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2027 - mse: 0.2027 - val_loss: 0.1136 - val_mse: 0.1136
Epoch 5/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1561 - mse: 0.1561 - val_loss: 0.1109 - val_mse: 0.1109
Epoch 6/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1366 - mse: 0.1366 - val_loss: 0.1061 - val_mse: 0.1061
Epoch 7/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1070 - mse: 0.1070 - val_loss: 0.1045 - val_mse: 0.1045
Epoch 8/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1427 - mse: 0.1427 - val_loss: 0.1011 - val_mse: 0.1011
Epoch 9/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.087

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [21]:
# Evaluate the model on training data
train_loss, train_mse = normalized_model.evaluate(X_train, y_train_scaled)


33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 840us/step - loss: 0.0105 - mse: 0.0105


- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [22]:
# Evaluate the model on validate data
val_loss, val_mse = normalized_model.evaluate(X_val, y_val_scaled)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1111 - mse: 0.1111 


Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [23]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)
y_val_original = ss_y.inverse_transform(y_val_scaled)

# RMSE of validate data
val_rmse = np.sqrt(mean_squared_error(y_val_original, y_val_pred))
print(f'Validation RMSE: {val_rmse}')


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Validation RMSE: 25953.842816121058


Great. Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'` 
  
[Documentation on the He Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/HeNormal)

In [24]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, activation='relu',
                         input_shape=(X_train.shape[1],),
                         kernel_initializer=initializers.he_normal()))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD',
                loss='mse', 
                metrics=['mse'])
# Train the model
he_model.fit(X_train, 
              y_train_scaled, 
              batch_size=32, 
              epochs=150, 
              validation_data=(X_val, y_val_scaled))


Epoch 1/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6256 - mse: 0.6256 - val_loss: 0.3115 - val_mse: 0.3115
Epoch 2/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2414 - mse: 0.2414 - val_loss: 0.2017 - val_mse: 0.2017
Epoch 3/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1708 - mse: 0.1708 - val_loss: 0.1853 - val_mse: 0.1853
Epoch 4/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1792 - mse: 0.1792 - val_loss: 0.1643 - val_mse: 0.1643
Epoch 5/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1470 - mse: 0.1470 - val_loss: 0.1706 - val_mse: 0.1706
Epoch 6/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1629 - mse: 0.1629 - val_loss: 0.1600 - val_mse: 0.1600
Epoch 7/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1262 - mse: 0.1262 - val_loss: 0.1563 - val_mse: 0.1563
Epoch 8/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1324 - mse: 0.1324 - val_loss: 0.1563 - val_mse: 0.1563
Epoch 9/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.125

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [25]:
# Evaluate the model on training data
train_loss, train_mse = he_model.evaluate(X_train, y_train_scaled)


33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 807us/step - loss: 0.0115 - mse: 0.0115


Evaluate the model (`he_model`) on validate data (`X_val` and `y_val_scaled`)

In [52]:
# Evaluate the model on validate data
val_loss, val_mse = he_model.evaluate(X_val, y_val_scaled)



9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2615 - mse: 0.2615 


## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 
  
[Documentation on the Lecun Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/LecunNormal)

In [37]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, kernel_initializer='lecun_normal', activation='relu', input_shape=(X_train.shape[1],)))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD',
                   loss='mse', 
                   metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6658 - mse: 0.6658 - val_loss: 0.1900 - val_mse: 0.1900
Epoch 2/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2145 - mse: 0.2145 - val_loss: 0.1440 - val_mse: 0.1440
Epoch 3/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1679 - mse: 0.1679 - val_loss: 0.1335 - val_mse: 0.1335
Epoch 4/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1668 - mse: 0.1668 - val_loss: 0.1211 - val_mse: 0.1211
Epoch 5/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1602 - mse: 0.1602 - val_loss: 0.1134 - val_mse: 0.1134
Epoch 6/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1081 - mse: 0.1081 - val_loss: 0.1085 - val_mse: 0.1085
Epoch 7/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1042 - mse: 0.1042 - val_loss: 0.1060 - val_mse: 0.1060
Epoch 8/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1115 - mse: 0.1115 - val_loss: 0.1003 - val_mse: 0.1003
Epoch 9/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.100

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [38]:
# Evaluate the model on training data
train_loss, train_mse = lecun_model.evaluate(X_train, y_train_scaled)

33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 722us/step - loss: 0.0076 - mse: 0.0076


Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [53]:
# Evaluate the model on validate data
val_loss, val_mse = lecun_model.evaluate(X_val, y_val_scaled)


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1214 - mse: 0.1214 


Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

[Documentation on the RMS Prop Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/experimental/RMSprop)

In [40]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(X_train.shape[1],)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer=optimizers.RMSprop(learning_rate=0.001),
                      loss='mse', 
                      metrics=['mse'])
# Code here

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.4873 - mse: 0.4873 - val_loss: 0.1539 - val_mse: 0.1539
Epoch 2/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2251 - mse: 0.2251 - val_loss: 0.1193 - val_mse: 0.1193
Epoch 3/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1513 - mse: 0.1513 - val_loss: 0.1012 - val_mse: 0.1012
Epoch 4/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1246 - mse: 0.1246 - val_loss: 0.1593 - val_mse: 0.1593
Epoch 5/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0763 - mse: 0.0763 - val_loss: 0.1138 - val_mse: 0.1138
Epoch 6/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0766 - mse: 0.0766 - val_loss: 0.1117 - val_mse: 0.1117
Epoch 7/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0566 - mse: 0.0566 - val_loss: 0.1342 - val_mse: 0.1342
Epoch 8/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0487 - mse: 0.0487 - val_loss: 0.1181 - val_mse: 0.1181
Epoch 9/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.038

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [41]:
# Evaluate the model on training data
train_loss, train_mse = rmsprop_model.evaluate(X_train, y_train_scaled)

33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 922us/step - loss: 0.0018 - mse: 0.0018


Evaluate the model (`rmsprop_model`) on training data (`X_val` and `y_val_scaled`) 

In [54]:
# Evaluate the model on validate data
val_loss, val_mse = rmsprop_model.evaluate(X_val, y_val_scaled)


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1069 - mse: 0.1069 


## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric

[Documentation on the Adam Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam)

In [43]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(X_train.shape[1],)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
# Code here
adam_model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                   loss='mse', 
                   metrics=['mse'])
# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.7971 - mse: 0.7971 - val_loss: 0.1348 - val_mse: 0.1348
Epoch 2/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1749 - mse: 0.1749 - val_loss: 0.1120 - val_mse: 0.1120
Epoch 3/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1063 - mse: 0.1063 - val_loss: 0.1387 - val_mse: 0.1387
Epoch 4/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0854 - mse: 0.0854 - val_loss: 0.1205 - val_mse: 0.1205
Epoch 5/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0556 - mse: 0.0556 - val_loss: 0.1121 - val_mse: 0.1121
Epoch 6/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0428 - mse: 0.0428 - val_loss: 0.1248 - val_mse: 0.1248
Epoch 7/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0425 - mse: 0.0425 - val_loss: 0.1237 - val_mse: 0.1237
Epoch 8/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0353 - mse: 0.0353 - val_loss: 0.1409 - val_mse: 0.1409
Epoch 9/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.035

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [44]:
# Evaluate the model on training data
train_loss, train_mse = adam_model.evaluate(X_train, y_train_scaled)

33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 826us/step - loss: 6.6918e-04 - mse: 6.6918e-04


Evaluate the model (`adam_model`) on training data (`X_val` and `y_val_scaled`) 

In [55]:
# Evaluate the model on validate data
val_loss, val_mse = adam_model.evaluate(X_val, y_val_scaled)


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1048 - mse: 0.1048 


## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [56]:
# Evaluate the best model on test data
test_loss, test_mse = adam_model.evaluate(X_test, ss_y.transform(y_test))

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1105 - mse: 0.1105 


As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [57]:
# Generate predictions on test data
y_test_pred_scaled = adam_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
test_mse = mean_squared_error(ss_y.inverse_transform(y_test), y_test_pred)
print(f'Test MSE: {test_mse}')
# RMSE of test data
test_rmse = np.sqrt(test_mse)
print(f'Test RMSE: {test_rmse}')


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
Test MSE: 2.6293287314614904e+20
Test RMSE: 16215204998.585403


## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.